# Build a Retrieval-Augmented Generation (RAG) using LangChain and OpenAI API

https://python.langchain.com/docs/tutorials/rag/

Install dependencies

In [24]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [25]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip -qU install python-magic unstructured


Usage:   
  /Users/isaaczepeda/projects/ML/notebooks/venv/bin/python -m pip <command> [options]

no such option: -U
Note: you may need to restart the kernel to use updated packages.


In [27]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [28]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [29]:
loader = DirectoryLoader("./data/", glob="**/*.txt")
docs = loader.load()
len(docs)

33

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [31]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

/Users/isaaczepeda/projects/ML/notebooks/venv/lib/python3.9/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [32]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [33]:
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)


In [39]:
rag_chain.invoke("como se asegura la justicia con la elección de jueces?")

'La justicia se asegura con la elección de jueces a través de un proceso abierto, transparente y público, donde la ciudadanía decide quiénes ocuparán estos cargos. Esta reforma permitirá que se elijan no solo jueces, sino también magistrados y ministros, promoviendo la probidad y el cumplimiento de requisitos específicos. Al involucrar al pueblo en la selección, se espera mejorar el Estado de derecho en el país.'